<a href="https://colab.research.google.com/github/bhartiansh/pruning_comparison_research/blob/main/TinyImageNET_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/spring2019/week03_convnets/tiny_img.py -O tiny_img.py

--2025-04-12 10:08:37--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/spring2019/week03_convnets/tiny_img.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3378 (3.3K) [text/plain]
Saving to: ‘tiny_img.py’

tiny_img.py         100%[===================>]   3.30K  --.-KB/s    in 0s      

2025-04-12 10:08:38 (58.3 MB/s) - ‘tiny_img.py’ saved [3378/3378]



In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!git clone https://github.com/bhartiansh/pruning_comparison_research
%cd pruning_comparison_research

Cloning into 'pruning_comparison_research'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 5.43 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/pruning_comparison_research


In [2]:
def load_tiny_imagenet_data(data_dir='./tiny-imagenet-200/tiny-imagenet-200'):
    train_dir = f'{data_dir}/train'
    val_dir = f'{data_dir}/val'

    # Validation folder fix (TinyImageNet stores val images differently)
    import os
    import shutil
    import pandas as pd

    val_annotations_path = os.path.join(val_dir, 'val_annotations.txt')
    val_img_dir = os.path.join(val_dir, 'images')
    val_data = pd.read_csv(val_annotations_path, sep='\t', header=None)
    val_data.columns = ['img', 'label', 'x1', 'y1', 'x2', 'y2']

    for label in val_data['label'].unique():
        os.makedirs(os.path.join(val_dir, label), exist_ok=True)

    for idx, row in val_data.iterrows():
        src = os.path.join(val_img_dir, row['img'])
        dst = os.path.join(val_dir, row['label'], row['img'])
        if os.path.exists(src):
            shutil.move(src, dst)

    if os.path.exists(val_img_dir):
        shutil.rmtree(val_img_dir)

    # Image generators
    train_gen = ImageDataGenerator(rescale=1./255)
    val_gen = ImageDataGenerator(rescale=1./255)

    train_generator = train_gen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical'
    )

    val_generator = val_gen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical'
    )

    return train_generator, val_generator